<a href="https://colab.research.google.com/github/Good-lei/AlphaGenome/blob/main/AlphaGenome-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 模型测试

In [1]:
from IPython.display import clear_output

In [2]:
!pip install alphagenome

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.4/174.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 6.2 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=851d259419c6a40739fc8a6f343917ce131ed9ebe9428ba737e7c2d246049a6d
  Stored in directory: /root/.cache/pip/wheels/65/c3/c3/238bf93c243597857edd94ddb0577faa74a8e16e9585896e83
Successfully built intervaltree


In [3]:
clear_output()

In [4]:
from alphagenome import colab_utils
from alphagenome.data import gene_annotation
from alphagenome.data import genome
from alphagenome.data import transcript as transcript_utils
from alphagenome.interpretation import ism
from alphagenome.models import dna_client
from alphagenome.models import variant_scorers
from alphagenome.visualization import plot_components
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
dna_model = dna_client.create("API-KEY")

In [6]:
[output.name for output in dna_client.OutputType]

['ATAC',
 'CAGE',
 'DNASE',
 'RNA_SEQ',
 'CHIP_HISTONE',
 'CHIP_TF',
 'SPLICE_SITES',
 'SPLICE_SITE_USAGE',
 'SPLICE_JUNCTIONS',
 'CONTACT_MAPS',
 'PROCAP']

## 模型应用

In [7]:
from alphagenome.data import genome
from alphagenome.models import dna_client
import numpy as np

API_KEY = "API-KEY"
model = dna_client.create(API_KEY)
print(model)

In [8]:
orig = genome.Interval("chr19", 17642000, 17643000, strand="-")

# 选择目标输入长度并扩展
L = dna_client.SEQUENCE_LENGTH_1MB
iv = orig.resize(L)  # 扩到 1,048,576bp 的区间

print(iv)

chr19:17118212-18166788:-


In [9]:
# 预测（按脑组织）
out = model.predict_interval(
    interval=iv,
    requested_outputs=[dna_client.OutputType.SPLICE_SITES,
                       dna_client.OutputType.SPLICE_SITE_USAGE],
    ontology_terms=["UBERON:0000955"],
)
#print(out)

In [10]:
output_metadata = model.output_metadata(
    organism=dna_client.Organism.HOMO_SAPIENS
)
#type(output_metadata)
#print(output_metadata)

In [11]:
# 1. 检查是否存在 'splice_sites' 属性
if hasattr(output_metadata, 'splice_sites'):
  print("✅ 成功找到 'splice_sites' 元数据：\n")
  # 2. 访问该属性，它是一个 pandas.DataFrame
  splice_sites_df = output_metadata.splice_sites
  # 3. 打印这个 DataFrame，这就是 'SpliceSite' 的所有说明
  print(splice_sites_df)

  # 您也可以打印 DataFrame 的信息来查看所有可用的“说明”列
  #print("\n--- DataFrame 的列信息 (说明) ---")
  #splice_sites_df.info()

else:
  print("❌ 未在 output_metadata 中找到 'splice_sites' 属性。")


#### 元数据输出为四行两列
#### 对应原始输出的四列信息

✅ 成功找到 'splice_sites' 元数据：

       name strand
0     donor      +
1  acceptor      +
2     donor      -
3  acceptor      -


In [12]:
# 1. 检查是否存在 'splice_sites' 属性
if hasattr(output_metadata, 'splice_site_usage'):
  print("✅ 成功找到 'splice_site_usage' 元数据：\n")
  # 2. 访问该属性，它是一个 pandas.DataFrame
  splice_site_usage_df = output_metadata.splice_site_usage
  # 3. 打印这个 DataFrame，这就是 'SpliceSite' 的所有说明
  print(splice_site_usage_df)

  # 您也可以打印 DataFrame 的信息来查看所有可用的“说明”列
  print("\n--- DataFrame 的列信息 (说明) ---")
  splice_site_usage_df.info()

else:
  print("❌ 未在 output_metadata 中找到 'splice_site_usage' 属性。")

✅ 成功找到 'splice_site_usage' 元数据：

                                                  name strand  \
0                  usage_CL:0000047 polyA plus RNA-seq      +   
1                       usage_CL:0000062 total RNA-seq      +   
2                  usage_CL:0000084 polyA plus RNA-seq      +   
3                       usage_CL:0000084 total RNA-seq      +   
4                       usage_CL:0000100 total RNA-seq      +   
..                                                 ...    ...   
729            usage_UBERON:0018116 polyA plus RNA-seq      -   
730            usage_UBERON:0018117 polyA plus RNA-seq      -   
731            usage_UBERON:0018118 polyA plus RNA-seq      -   
732  usage_UBERON:0036149 gtex Skin_Not_Sun_Exposed...      -   
733                 usage_UBERON:0036149 total RNA-seq      -   

            Assay title  ontology_curie                   biosample_name  \
0    polyA plus RNA-seq      CL:0000047               neuronal stem cell   
1         total RNA-seq      CL:00

In [13]:
# 将全长输出切回到你的原窗口（坐标对齐，无需手工反向互补）
sites_sub = out.splice_sites.slice_by_interval(orig, match_resolution=True).values
usage_sub = out.splice_site_usage.slice_by_interval(orig, match_resolution=True).values

In [14]:
print(sites_sub.shape)
print(sites_sub)

#“donor+/donor−/acceptor+/acceptor−”
#正链供体，正链受体，负链供体，负链受体的概率

(1000, 4)
[[7.1153045e-07 3.1478703e-07 1.9967556e-06 8.8661909e-07]
 [4.3213367e-07 3.7997961e-07 1.4603138e-06 2.4884939e-06]
 [9.4249845e-07 1.9185245e-07 2.8684735e-07 4.3511391e-06]
 ...
 [3.1478703e-07 1.4901161e-07 8.3073974e-07 5.3644180e-07]
 [4.6193600e-06 1.9371510e-06 5.4240227e-06 1.1444092e-03]
 [1.6856939e-07 2.9616058e-07 4.7311187e-07 8.5681677e-07]]


In [15]:
col = 2  # 指定列，如第3列
idx = np.argsort(sites_sub[:, col])[::-1]   # 原始行索引，按该列从大到小
rows_sorted = sites_sub[idx]                # 排序后的行

# 取前N个
N = 10
top_idx = idx[:N]

print("row\tval_col2")
for r in top_idx:
    print(f"{r}\t{sites_sub[r, col]:.6f}")

row	val_col2
844	0.988281
556	0.099121
413	0.067383
883	0.012329
838	0.006195
610	0.004517
480	0.002838
635	0.002441
381	0.001480
863	0.001137


In [16]:
print(usage_sub.shape)
print(usage_sub)

#正链剪接位点使用率，负链剪接位点使用率

(1000, 2)
[[3.2931566e-06 4.7981739e-06]
 [2.9057264e-06 4.2319298e-06]
 [3.2931566e-06 5.4240227e-06]
 ...
 [6.8917871e-07 1.2069941e-06]
 [8.8661909e-07 5.4240227e-06]
 [7.8231096e-07 1.2069941e-06]]


In [17]:
col = 1  # 指定列，如第2列
idx = np.argsort(usage_sub[:, col])[::-1]   # 原始行索引，按该列从大到小
rows_sorted = usage_sub[idx]                # 排序后的行

# 取前N个
N = 10
top_idx = idx[:N]

print("row\tval_col2")
for r in top_idx:
    print(f"{r}\t{usage_sub[r, col]:.6f}")

row	val_col2
844	0.839844
959	0.835938
845	0.000357
590	0.000191
958	0.000179
413	0.000149
556	0.000109
540	0.000075
894	0.000058
847	0.000051


## 预测变异前后的变化

In [18]:
from alphagenome.data import genome
from alphagenome.models import dna_client
import numpy as np

# 已有：API_KEY, model, orig, L, iv, ontology_terms
ontology_terms = ["UBERON:0000955"]

orig = genome.Interval("chr19", 17642000, 17643000, strand="-")

# 选择目标输入长度并扩展
L = dna_client.SEQUENCE_LENGTH_1MB
iv = orig.resize(L)  # 扩到 1,048,576bp 的区间


# 1) 定义变异：示例 SNV，请替换为你的坐标与碱基
var = genome.Variant(
    chromosome="chr19",
    position=17642430,       # 变异坐标（基因组坐标，跟 strand 无关）
    reference_bases="C",
    alternate_bases="G",
)

# 2) 调用变异预测：一次返回 REF 与 ALT 两套轨道
res = model.predict_variant(
    interval=iv,
    variant=var,
    requested_outputs=[
        dna_client.OutputType.SPLICE_SITES,
        dna_client.OutputType.SPLICE_SITE_USAGE,
    ],
    ontology_terms=ontology_terms,
)

# 3) 切回你的原窗口（1000bp）
ref_sites  = res.reference.splice_sites.slice_by_interval(orig, match_resolution=True).values
alt_sites  = res.alternate.splice_sites.slice_by_interval(orig, match_resolution=True).values
ref_usage  = res.reference.splice_site_usage.slice_by_interval(orig, match_resolution=True).values
alt_usage  = res.alternate.splice_site_usage.slice_by_interval(orig, match_resolution=True).values


In [19]:
print(ref_usage.shape)
print(ref_usage)

(1000, 2)
[[3.2931566e-06 4.7981739e-06]
 [2.9057264e-06 4.2319298e-06]
 [3.2931566e-06 5.4240227e-06]
 ...
 [6.8917871e-07 1.2069941e-06]
 [8.8661909e-07 5.4240227e-06]
 [7.8231096e-07 1.2069941e-06]]


In [20]:
col = 1  # 指定列，如第2列
idx = np.argsort(alt_usage[:, col])[::-1]   # 原始行索引，按该列从大到小
rows_sorted = alt_usage[idx]                # 排序后的行

# 取前N个
N = 10
top_idx = idx[:N]

print("row\tval_col2")
for r in top_idx:
    print(f"{r}\t{alt_usage[r, col]:.6f}")

row	val_col2
959	0.839844
844	0.835938
845	0.000368
590	0.000203
958	0.000179
413	0.000179
556	0.000109
540	0.000085
894	0.000058
847	0.000051


In [22]:
# 5) 计算变化：Δusage = ALT - REF
d_usage = alt_usage - ref_usage

# 6) 例：按第2列的 |Δusage| 从大到小输出行索引与数值（你之前的格式）
col = 1
idx = np.argsort(np.abs(d_usage[:, col]))[::-1]
N = 50
hdr = f"{'row':>6} {'pos':>10} {'Δusage':>12} {'REF':>12} {'ALT':>12}"
print(hdr)
print("-" * len(hdr))

for r in idx[:N]:
    pos = orig.start + r
    print(f"{r:>6d} {pos:>10d} {d_usage[r,col]:>+12.6f} {ref_usage[r,col]:>12.6f} {alt_usage[r,col]:>12.6f}")

   row        pos       Δusage          REF          ALT
--------------------------------------------------------
   959   17642959    +0.003906     0.835938     0.839844
   844   17642844    -0.003906     0.839844     0.835938
   413   17642413    +0.000031     0.000149     0.000179
   590   17642590    +0.000012     0.000191     0.000203
   845   17642845    +0.000011     0.000357     0.000368
   540   17642540    +0.000010     0.000075     0.000085
   504   17642504    +0.000005     0.000024     0.000029
   455   17642455    +0.000005     0.000023     0.000028
   887   17642887    -0.000003     0.000048     0.000045
   946   17642946    +0.000003     0.000040     0.000043
   609   17642609    +0.000002     0.000033     0.000035
   431   17642431    +0.000002     0.000006     0.000008
   850   17642850    +0.000002     0.000031     0.000033
   944   17642944    -0.000002     0.000031     0.000029
   884   17642884    +0.000002     0.000028     0.000029
   853   17642853    +0.000002 